<a href="https://colab.research.google.com/github/LugenderGeist/computer_vision_3/blob/main/computer_vision_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №3  
Целью лабораторной работы является нахождение движущихся объектов на видео. Для этого необходимо:
- Обработать кадры видео с помощью фильтров
- Вычислить среднее значение пикселей на видео
- Создать изображение со средними значениями пикселей, чтобы определить фон
- Вычесть фон из каждого кадра, чтобы найти отличия: разность между пикселями считается нулевой при заданном пороге, при переходе через порог - пиксели относятся к искомому объекту
- Получить бинаризованные изображения, где видно объект
- Найденный объект выделить прямоугольником, можно через опен сиви
- Из полученных бинаризованных кадров сложить видео

In [2]:
import cv2
import numpy as np
from google.colab import drive
from IPython.display import clear_output, Image, display
from google.colab.patches import cv2_imshow

drive.mount('/content/drive')

Mounted at /content/drive


Сначала выведем видео, на котором grayscale останется только на движущемся объекте, а фон станет черным (так проще визуально понять, как определяется объект). Изменяя параметры и применяя морфологические операции и фильтр Гаусса приблизим распознавание к желаемому.

In [28]:
# Параметры обработки видео
params = {
    "input_file": '/content/drive/MyDrive/Colab Notebooks/computer_vision/video.mp4',  # Путь к входному видео
    "output_file": "output_video.mp4",  # Имя выходного файла
    "codec": 'mp4v',  # Кодек для записи видео
    "isColor": False,  # Цветность выходного видео (False для grayscale)
    "background_subtractor": {
        "history": 1000,  # Размер истории для модели фона
        "varThreshold": 520,  # Порог чувствительности
        "detectShadows": True  # Обнаружение теней
    },
    "morphology": {
        "kernel_size": (5, 5),  # Размер ядра для морфологических операций
        "apply_open": True,  # Будет ли применено MORPH_OPEN
        "apply_close": False  # Будет ли применено MORPH_CLOSE
    },
    "blur": {
        "apply": False,  # Нужен ли фильтр Гаусса
        "kernel_size": (3, 3)  # Размер ядра для размытия
    }
}

cap = cv2.VideoCapture(params["input_file"])

# Получаем параметры исходного видео
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)  # Количество кадров в секунду

# Определяем кодек и создаем объект VideoWriter для записи видео
fourcc = cv2.VideoWriter_fourcc(*params["codec"])
out = cv2.VideoWriter(params["output_file"], fourcc, fps, (frame_width, frame_height), params["isColor"])

# Создаем объект для вычитания фона
fgbg = cv2.createBackgroundSubtractorMOG2(
    history=params["background_subtractor"]["history"],
    varThreshold=params["background_subtractor"]["varThreshold"],
    detectShadows=params["background_subtractor"]["detectShadows"]
)

# Инициализируем счетчик кадров
frame_count = 0

# Цикл для чтения и обработки каждого кадра
while True:
    ret, frame = cap.read()
    if not ret:
        print("Видео завершено")
        break

    frame_count += 1

    # Каждый кадр переводим в grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Применяем вычитание фона для создания маски
    fg_mask = fgbg.apply(gray_frame)

    # Морфологические операции
    if params["morphology"]["apply_open"] or params["morphology"]["apply_close"]:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, params["morphology"]["kernel_size"])
        if params["morphology"]["apply_open"]:
            fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)  # Удаляем шумы
        if params["morphology"]["apply_close"]:
            fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)  # Заполняем пробелы

    # Фильтр Гаусса
    if params["blur"]["apply"]:
        fg_mask = cv2.GaussianBlur(fg_mask, params["blur"]["kernel_size"], 0)

    # Накладываем маску для выделения движущихся объектов
    motion_frame = cv2.bitwise_and(gray_frame, gray_frame, mask=fg_mask)

    # Преобразуем кадр в 8-битное изображение
    motion_frame = np.uint8(motion_frame)

    # Записываем обработанный кадр в выходной файл
    out.write(motion_frame)

    # Очищаем вывод
    clear_output(wait=True)

# Освобождаем ресурсы
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Обработка завершена. Всего обработано кадров: {frame_count}")

# Скачивание видео
from google.colab import files
files.download(params["output_file"])

Видео завершено
Обработка завершена. Всего обработано кадров: 172


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

После того, как объект стал распознаваться достаточно хорошо - создаем рамку вокруг объекта.

In [30]:
# Параметры обработки видео
params["isColor"] = True

# Создаем объект VideoCapture
cap = cv2.VideoCapture(params["input_file"])

# Получаем параметры исходного видео
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)  # Количество кадров в секунду

# Определяем кодек и создаем объект VideoWriter для записи видео
fourcc = cv2.VideoWriter_fourcc(*params["codec"])
out = cv2.VideoWriter(params["output_file"], fourcc, fps, (frame_width, frame_height), params["isColor"])

# Создаем объект для вычитания фона
fgbg = cv2.createBackgroundSubtractorMOG2(
    history=params["background_subtractor"]["history"],
    varThreshold=params["background_subtractor"]["varThreshold"],
    detectShadows=params["background_subtractor"]["detectShadows"]
)

# Инициализируем счетчик кадров
frame_count = 0

# Цикл для чтения и обработки каждого кадра
while True:
    ret, frame = cap.read()
    if not ret:
        print("Видео завершено")
        break

    frame_count += 1

    # Каждый кадр переводим в grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Применяем вычитание фона для создания маски
    fg_mask = fgbg.apply(gray_frame)

    # Морфологические операции
    if params["morphology"]["apply_open"] or params["morphology"]["apply_close"]:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, params["morphology"]["kernel_size"])
        if params["morphology"]["apply_open"]:
            fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)  # Удаляем шумы
        if params["morphology"]["apply_close"]:
            fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)  # Заполняем пробелы

    # Находим контуры на маске
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Рисуем рамки вокруг найденных объектов
    for contour in contours:
        # Вычисляем площадь контура и игнорируем маленькие шумы
        if cv2.contourArea(contour) < 500:  # Минимальная площадь объекта
            continue

        # Вычисляем ограничивающий прямоугольник
        x, y, w, h = cv2.boundingRect(contour)

        # Рисуем прямоугольник на исходном кадре
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)

    # Записываем обработанный кадр в выходной файл
    out.write(frame)

    # Очищаем вывод
    clear_output(wait=True)

# Освобождаем ресурсы
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Обработка завершена. Всего обработано кадров: {frame_count}")

# Скачивание видео
from google.colab import files
files.download(params["output_file"])

Видео завершено
Обработка завершена. Всего обработано кадров: 172


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>